In [1]:
import pandas as pd
from binance.um_futures import UMFutures
from datetime import datetime, timedelta
import pytz

def get_binance_data(symbol="BTCUSDT", interval="5m", date="2025-06-01", duration_hours=24):
    """
    Get Binance futures kline data for a specific date and duration
    
    Args:
        symbol: Trading pair (default: "BTCUSDT")
        interval: Kline interval (default: "5m")
        date: Date in format "YYYY-MM-DD" (default: "2025-06-01")
        duration_hours: Duration in hours (default: 24 for full day)
    
    Returns:
        DataFrame with OHLCV data
    """
    um_futures_client = UMFutures()
    
    # Convert date string to datetime in UTC
    start_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = start_date.replace(tzinfo=pytz.UTC)  # Set timezone to UTC
    end_date = start_date + timedelta(hours=duration_hours)
    
    # Convert to milliseconds for Binance API
    start_time = int(start_date.timestamp() * 1000)
    end_time = int(end_date.timestamp() * 1000)
    
    # Get data from Binance
    data = um_futures_client.klines(
        symbol=symbol, 
        interval=interval, 
        startTime=start_time,
        endTime=end_time,
        limit=1000
    )
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=[
        "datetime", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
        "ignore"
    ])
    
    return df

# Get 5-minute data for June 1, 2025 (full day) starting from UTC 00:00:00
data = get_binance_data(interval="15m", date="2025-06-03", duration_hours=24)
df = data

# Convert the 'datetime' column to a datetime object
df["datetime"] = pd.to_datetime(df["datetime"], unit="ms")

# Keep only the relevant columns
df = df[["datetime", "open", "high", "low", "close", "volume"]]
df = df.set_index("datetime")

# save the DataFrame to a CSV file
df.to_csv("BTCUSDT_2025-06-04_15min.csv")

In [7]:
# df_from_csv = pd.read_csv("BTCUSDT_2025-06-01_5min.csv", parse_dates=True)
# df_from_csv

val=(24*60)/5
val

288.0

### Getting daily data including previously 21 candle

In [34]:
import pandas as pd
from binance.um_futures import UMFutures
from datetime import datetime, timedelta
import pytz


def get_binance_data(symbol="BTCUSDT", interval="15m", date="2025-06-03", duration_hours=24, lookback_candles=21):
    """
    Fetch Binance futures kline data based on UTC day boundaries, including lookback candles.

    Args:
        symbol: Trading pair (e.g., "BTCUSDT")
        interval: Kline interval (e.g., "15m")
        date: Target day in "YYYY-MM-DD" (UTC)
        duration_hours: How many hours of data to fetch from the date (default: 24)
        lookback_candles: Number of candles from the *previous UTC day* to include

    Returns:
        pd.DataFrame with OHLCV data from (previous 21 candles + full UTC day)
    """
    um_futures_client = UMFutures()

    # Map interval to minutes
    interval_map = {
        "1m": 1, "3m": 3, "5m": 5, "15m": 15, "30m": 30,
        "1h": 60, "2h": 120, "4h": 240, "6h": 360,
        "8h": 480, "12h": 720, "1d": 1440
    }

    if interval not in interval_map:
        raise ValueError(f"Unsupported interval '{interval}'")

    candle_minutes = interval_map[interval]
    lookback_minutes = lookback_candles * candle_minutes

    # Parse the given date in UTC (00:00:00)
    utc = pytz.UTC
    start_utc = utc.localize(datetime.strptime(
        date, "%Y-%m-%d"))  # 2025-06-03 00:00:00 UTC
    # 2025-06-04 00:00:00 UTC
    end_utc = start_utc + timedelta(hours=duration_hours)
    lookback_start_utc = start_utc - \
        timedelta(minutes=lookback_minutes)  # Go back 21 candles

    # Convert to ms timestamps
    start_time = int(lookback_start_utc.timestamp() * 1000)
    end_time = int(end_utc.timestamp() * 1000)

    # Fetch candles
    data = um_futures_client.klines(
        symbol=symbol,
        interval=interval,
        startTime=start_time,
        endTime=end_time,
        limit=1500
    )

    # Create DataFrame
    df = pd.DataFrame(data, columns=[
        "datetime", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
        "ignore"
    ])

    df["datetime"] = pd.to_datetime(df["datetime"], unit="ms", utc=True)
    df = df[["datetime", "open", "high", "low", "close", "volume"]]
    df.set_index("datetime", inplace=True)

    return df


# ✅ Example usage
df = get_binance_data(
    symbol="BTCUSDT",
    interval="15m",
    date="2025-05-01",  # Treated as UTC day
    duration_hours=24,
    lookback_candles=21
)
# Remove timezone info for CSV output (make datetime naive but still in UTC)
df.index = df.index.tz_convert(None)
df.to_csv("BTCUSDT_2025-05-01_UTC_with_lookback.csv")
print(df.head(25))

                         open      high       low     close    volume
datetime                                                             
2025-04-30 18:45:00  94020.20  94140.80  93985.30  94043.80   699.703
2025-04-30 19:00:00  94043.90  94129.50  93981.60  94049.90   585.920
2025-04-30 19:15:00  94050.00  94066.50  93700.00  93741.20  1428.713
2025-04-30 19:30:00  93741.20  93920.00  93715.60  93912.80   743.159
2025-04-30 19:45:00  93912.70  94229.90  93903.50  94118.20  2353.267
2025-04-30 20:00:00  94118.20  94653.80  94068.90  94596.80  3557.106
2025-04-30 20:15:00  94596.70  94720.40  94442.00  94670.00  2229.626
2025-04-30 20:30:00  94669.90  94689.80  94480.00  94558.40  1063.186
2025-04-30 20:45:00  94558.40  94667.70  94502.60  94515.90   820.162
2025-04-30 21:00:00  94515.90  94751.80  94436.40  94535.80  1510.550
2025-04-30 21:15:00  94535.90  94650.40  94535.80  94585.90   455.202
2025-04-30 21:30:00  94585.90  94609.00  94431.80  94431.80   531.254
2025-04-30 21:45:00 

## for daily,month or even yearly different timeframe candle data

In [1]:
import pandas as pd
from binance.um_futures import UMFutures
from datetime import datetime, timedelta
import pytz

def fetch_binance_klines(symbol="BTCUSDT", interval="15m", start_date="2025-01-01", end_date="2025-01-31"):
    """
    Fetch historical Binance Futures kline data between specified dates and interval.
    
    Args:
        symbol (str): Trading pair (e.g., "BTCUSDT")
        interval (str): Kline interval (e.g., "5m", "15m", "1h", etc.)
        start_date (str): Start date in "YYYY-MM-DD" format (UTC)
        end_date (str): End date in "YYYY-MM-DD" format (UTC)
        
    Returns:
        pd.DataFrame: DataFrame containing OHLCV data with datetime index
    """
    client = UMFutures()
    
    interval_map = {
        "1m": 1, "3m": 3, "5m": 5, "15m": 15, "30m": 30,
        "1h": 60, "2h": 120, "4h": 240, "6h": 360,
        "8h": 480, "12h": 720, "1d": 1440
    }
    
    if interval not in interval_map:
        raise ValueError(f"Unsupported interval '{interval}'")
    
    # Parse and localize dates
    utc = pytz.UTC
    start_dt = utc.localize(datetime.strptime(start_date, "%Y-%m-%d"))
    end_dt = utc.localize(datetime.strptime(end_date, "%Y-%m-%d"))
    
    df_list = []
    limit = 1500  # Max candles per request
    step_minutes = interval_map[interval] * limit
    step = timedelta(minutes=step_minutes)
    
    current_start = start_dt
    
    while current_start < end_dt:
        current_end = min(current_start + step, end_dt)
        
        start_ms = int(current_start.timestamp() * 1000)
        end_ms = int(current_end.timestamp() * 1000)
        
        klines = client.klines(
            symbol=symbol,
            interval=interval,
            startTime=start_ms,
            endTime=end_ms,
            limit=limit
        )
        
        if not klines:
            break
        
        temp_df = pd.DataFrame(klines, columns=[
            "datetime", "open", "high", "low", "close", "volume",
            "close_time", "quote_asset_volume", "number_of_trades",
            "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
        ])
        
        temp_df["datetime"] = pd.to_datetime(temp_df["datetime"], unit="ms", utc=True)
        temp_df = temp_df[["datetime", "open", "high", "low", "close", "volume"]]
        df_list.append(temp_df)
        
        current_start = current_end
    
    if df_list:
        full_df = pd.concat(df_list).drop_duplicates(subset=["datetime"]).reset_index(drop=True)
        full_df.set_index("datetime", inplace=True)
        return full_df
    else:
        print("No data retrieved.")
        return pd.DataFrame()
    
df = fetch_binance_klines(
    symbol="BTCUSDT",
    interval="15m",
    start_date="2025-01-01",
    end_date="2025-05-31"
)

# Remove timezone for CSV
df.index = df.index.tz_convert(None)
df.to_csv("BTCUSDT_FIVE2025_15m.csv")

# print(df.head())
df



,open,high,low,close,volume
datetime,,,,,
2025-01-01 00:00:00,93548.80,93690.00,93460.20,93631.20,988.149
2025-01-01 00:15:00,93631.20,93885.00,93588.00,93743.30,1073.305
2025-01-01 00:30:00,93743.40,93948.70,93743.30,93857.10,1181.898
2025-01-01 00:45:00,93857.10,94449.20,93834.80,94363.60,2501.257
2025-01-01 01:00:00,94363.60,94363.70,93882.70,94116.30,1675.921
...,...,...,...,...,...
2025-05-30 23:00:00,103920.00,104210.20,103698.60,103981.20,2466.206
2025-05-30 23:15:00,103981.20,104171.20,103605.60,103805.30,2967.367
2025-05-30 23:30:00,103805.40,103883.20,103555.00,103756.10,2731.237
